Name: Kanchika Sudhirkumar Kapoor

Email Id: 10621287@mydbs.ie

Colab Link: https://colab.research.google.com/drive/1gdmtz4r5WuluAC3S8SC9zC0Lm-tiiGtj

In [ ]:
"""
Primary objective: To design and develop a Data Acquisition and Preprocessing Pipeline.

You are required to develop a Data Acquisition and Preprocessing Pipeline of your choice, including data acquisition (API, Web scraping, DB Extract etc.), Extraction of features and Transformations as appropriate, followed by loading into an appropriate database. The focus of the complexity of the pipeline is your choice.

Use Git (e.g. GitHub / Colab) and commit/pin version regularly.

    GitHub/GitLab/Colab MUST be used for this project to develop both the artefact and documentation, and any code or material uploaded as a fait accompli will not be credited. Furthermore, any such code not attributed or presented contrary to its originating licence will be the subject of Academic Impropriety investigations.
    If using Colab, it must be shared with paul.laird@dbs.ie as editor, and documentation should be as text cells in line with the code. If using Git, documentation must be in the form of a text document (README.md / HTML / LaTeX) inside the git repository, and the repository must be public or shared with paul-laird
    The repository must be regularly active from week 7.

Demonstration will be on Tuesday Wk 12, with submission on the Sunday beforehand

Implementation 70%

Documentation 30%
"""

### Data Source:
The data which is used for this assessment is from following page:

https://ycharts.com/events/calendar/#/?date=2022-11-29&pageNum=1&eventGroups=earnings,dividends,splits_spinoffs,other&securitylistName=all_stocks&securityGroup=company&viewMode=week

It lists all the quarterly earning calls and results by various companies during a given date range

Requests library is used for fetching data, Pandas is used for data wrangling and SQLalchemy is used for connecting and storing the data in sql database

First, we install necessary packages and import various modules

In [66]:
%%sh
curl https://packages.microsoft.com/keys/microsoft.asc | apt-key add -
curl https://packages.microsoft.com/config/ubuntu/18.04/prod.list > /etc/apt/sources.list.d/mssql-release.list
sudo apt-get update
sudo ACCEPT_EULA=Y apt-get -q -y install msodbcsql17

OK
Hit:1 http://archive.ubuntu.com/ubuntu bionic InRelease
Hit:2 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease
Hit:3 https://packages.microsoft.com/ubuntu/18.04/prod bionic InRelease
Get:4 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Get:5 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [83.3 kB]
Get:6 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Ign:7 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:8 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Hit:9 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Hit:10 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease
Hit:11 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Hit:12 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease
Hit:14 http://ppa.launchpad.net/graphics-drivers/ppa

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100   983  100   983    0     0  15359      0 --:--:-- --:--:-- --:--:-- 15359
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100    77  100    77    0     0   2750      0 --:--:-- --:--:-- --:--:--  2750


In [67]:
!pip install pypyodbc

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [68]:
import requests
from io import StringIO
import pandas as pd
import json
from urllib.parse import urlencode

Here, the url is specified and the `GET` request is used. This block is to check and observe the data

In [69]:
url = 'https://ycharts.com/events/events_data/json?date=11/01/2021&startDate=11/25/2022&endDate=12/01/2022&pageNum=5&eventGroups=earnings,dividends,splits_spinoffs,other&securitylistName=all_stocks&securityGroup=company&sort=asc'

header = {'Accept': 'application/json',
          'Content':'application/json',
          'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:107.0) Gecko/20100101 Firefox/107.0'
          }
r = requests.get(url, headers=header)
r.content

b'{"events": [{"description": "0.236 Normal dividend, declared October 14, 2021 and paid November 15, 2021", "event_date": "2021-11-01", "event_date_fmtd": "11/01/2021", "event_date_js": 1635724800000.0, "event_datetime_utc_js": null, "event_subgroup": "dividends", "event_subgroup_display": "Dividends", "event_time_fmtd": null, "is_today_or_later": false, "is_estimate": false, "security_id": "O", "security_name": "Realty Income Corp", "security_url": "/companies/O", "currency_code": "USD", "extra": {"amount": 0.2360000000, "dividend_type": "Normal", "ex_date_fmtd": "11/01/2021", "pay_date_fmtd": "11/15/2021", "declared_date_fmtd": "10/14/2021"}}, {"description": "0.70 Normal dividend, declared October 20, 2021 and paid November 09, 2021", "event_date": "2021-11-01", "event_date_fmtd": "11/01/2021", "event_date_js": 1635724800000.0, "event_datetime_utc_js": null, "event_subgroup": "dividends", "event_subgroup_display": "Dividends", "event_time_fmtd": null, "is_today_or_later": false, "i

### API Observations:
* The data received is paginated. A loop can be used for getting all the data. This can then be appended to an array and processed further

* To get data for a particular day, `date` query param is used and to get data for a range, `startDate` and `endDate` is used. If all 3 are provided, then api responds back with `date` data,i.e, date has precedance over date range.
To get date range data, `date` param needs to be omitted

* The date format used by api is in format: `MM/DD/YYYY`.

* The api blocks request if no `User-Agent` is provided in headers


---



In following blocks, the data will be fetched using api and will be transformed to dataframe

In [70]:
def dict_to_query_params(query_params):
  """
  method for dictionary to query param convertion
  returns url encoded string
  """
  return urlencode(query_params)

start_date = '11/20/2022'
end_date = '12/01/2022'
page_num = 1

params = {
     'startDate': start_date,
     'endDate':end_date,
     'pageNum':page_num,
     'eventGroups':'earnings,dividends,splits_spinoffs,other',
     'securitylistName':'all_stocks',
     'securityGroup':'company',
     'sort':'asc'
  }

base_url = 'https://ycharts.com/events/events_data/json'

url = base_url+'?'+dict_to_query_params(params)
url

'https://ycharts.com/events/events_data/json?startDate=11%2F20%2F2022&endDate=12%2F01%2F2022&pageNum=1&eventGroups=earnings%2Cdividends%2Csplits_spinoffs%2Cother&securitylistName=all_stocks&securityGroup=company&sort=asc'

In [71]:
def get_json_response(url, headers):
  """
  calls the url with given headers using get
  returns json response
  """
  try:
    resp = requests.get(url, headers=headers)
    data = resp.json()
    return data
  except Exception as e:
    print('error: '+str(e))

data = get_json_response(url, header)
data

{'events': [{'description': 'Q4 2022 Earnings Results',
   'event_date': '2022-11-20',
   'event_date_fmtd': '11/20/2022',
   'event_date_js': 1668902400000.0,
   'event_datetime_utc_js': None,
   'event_subgroup': 'earnings_results',
   'event_subgroup_display': 'Earnings Results',
   'event_time_fmtd': None,
   'is_today_or_later': False,
   'is_estimate': False,
   'security_id': 'CBBYF',
   'security_name': 'Virgin Money UK PLC',
   'security_url': '/companies/CBBYF',
   'extra': {'fiscal_quarter': 'Q4',
    'fiscal_year': 2022,
    'estimate_data': None}},
  {'description': 'Q4 2022 Earnings Results',
   'event_date': '2022-11-20',
   'event_date_fmtd': '11/20/2022',
   'event_date_js': 1668902400000.0,
   'event_datetime_utc_js': None,
   'event_subgroup': 'earnings_results',
   'event_subgroup_display': 'Earnings Results',
   'event_time_fmtd': None,
   'is_today_or_later': False,
   'is_estimate': False,
   'security_id': 'CYBBF',
   'security_name': 'Virgin Money UK PLC',
   '

In [72]:
def get_paginated_data_arr(base_url,headers,params,data_key_name,total_pages_num,curr_page_num_key):
  """
  loops till total_pages_num and fetches data for every page
  returns a list containing all data
  """
  try:
    total_arr = []
    for i in range(1, total_pages_num+1):
      url = base_url + dict_to_query_params(params)
      data = get_json_response(url, headers)[data_key_name]
      total_arr += data 
      params[curr_page_num_key]+=1
    return total_arr
  except Exception as e:
    print('error: '+str(e))

total_pages = data['paginationInfo']['num_pages']
json_data = get_paginated_data_arr(base_url+'?', header, params, 'events', total_pages, 'pageNum')

In [73]:
#check if len of ceated json data equals 'num_items' in pagination info in response
len(json_data) == data['paginationInfo']['num_items']

True

### Data Transformation:
Convert json data to dataframe using pandas `read_json` method.

Latest version of pandas requires json data to be converted into buffer of stringified json for `read_json` method to work

In [74]:
json_str = json.dumps(json_data) #convert dictionary to string
json_buf = StringIO(json_str) # make string buffer

df = pd.read_json(json_buf)
df

,description,event_date,event_date_fmtd,event_date_js,event_datetime_utc_js,event_subgroup,event_subgroup_display,event_time_fmtd,is_today_or_later,is_estimate,security_id,security_name,security_url,extra,currency_code
0,Q4 2022 Earnings Results,2022-11-20,11/20/2022,1668902400000,NaN,earnings_results,Earnings Results,None,False,False,CBBYF,Virgin Money UK PLC,/companies/CBBYF,"{'fiscal_quarter': 'Q4', 'fiscal_year': 2022, ...",NaN
1,Q4 2022 Earnings Results,2022-11-20,11/20/2022,1668902400000,NaN,earnings_results,Earnings Results,None,False,False,CYBBF,Virgin Money UK PLC,/companies/CYBBF,"{'fiscal_quarter': 'Q4', 'fiscal_year': 2022, ...",NaN
2,Q4 2022 Earnings Results,2022-11-21,11/21/2022,1668988800000,NaN,earnings_results,Earnings Results,None,False,False,A,Agilent Technologies Inc,/companies/A,"{'fiscal_quarter': 'Q4', 'fiscal_year': 2022, ...",NaN
3,Q4 2022 Earnings Call,2022-11-21,11/21/2022,1668988800000,1.669066e+12,earnings_calls,Earnings Calls,16:30 EST,False,False,A,Agilent Technologies Inc,/companies/A,"{'fiscal_quarter': 'Q4', 'fiscal_year': 2022, ...",NaN
4,Q3 2022 Earnings Results,2022-11-21,11/21/2022,1668988800000,NaN,earnings_results,Earnings Results,None,False,False,ACAQ,Athena Consumer Acquisition Corp,/companies/ACAQ,"{'fiscal_quarter': 'Q3', 'fiscal_year': 2022, ...",NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1659,333333 to 10000000 Split,2022-12-01,12/01/2022,1669852800000,NaN,splits,Splits,None,False,False,TUEM,Tuesday Morning Corp,/companies/TUEM,{'ratio': '333333:10000000'},NaN
1660,1546061 to 10000000 Split,2022-12-01,12/01/2022,1669852800000,NaN,splits,Splits,None,False,False,VSQTF,Victory Square Technologies Inc,/companies/VSQTF,{'ratio': '1546061:10000000'},NaN
1661,1546061 to 10000000 Split,2022-12-01,12/01/2022,1669852800000,NaN,splits,Splits,None,False,False,VST.CX,Victory Square Technologies Inc,/companies/VST.CX,{'ratio': '1546061:10000000'},NaN
1662,208059 to 200000 Split,2022-12-01,12/01/2022,1669852800000,NaN,spinoffs,Spinoffs,None,False,False,FNF,Fidelity National Financial Inc,/companies/FNF,{'ratio': '208059:200000'},NaN


### Wrangle Data
`event_date`,	`event_date_fmtd`, 	`event_date_js`, `event_datetime_utc_js` columns specify the same date in different formats.
We can retain `event_date` column and drop others

In [75]:
df = df.drop(['event_date_fmtd', 'event_date_js', 'event_datetime_utc_js'], axis=1)
df

,description,event_date,event_subgroup,event_subgroup_display,event_time_fmtd,is_today_or_later,is_estimate,security_id,security_name,security_url,extra,currency_code
0,Q4 2022 Earnings Results,2022-11-20,earnings_results,Earnings Results,None,False,False,CBBYF,Virgin Money UK PLC,/companies/CBBYF,"{'fiscal_quarter': 'Q4', 'fiscal_year': 2022, ...",NaN
1,Q4 2022 Earnings Results,2022-11-20,earnings_results,Earnings Results,None,False,False,CYBBF,Virgin Money UK PLC,/companies/CYBBF,"{'fiscal_quarter': 'Q4', 'fiscal_year': 2022, ...",NaN
2,Q4 2022 Earnings Results,2022-11-21,earnings_results,Earnings Results,None,False,False,A,Agilent Technologies Inc,/companies/A,"{'fiscal_quarter': 'Q4', 'fiscal_year': 2022, ...",NaN
3,Q4 2022 Earnings Call,2022-11-21,earnings_calls,Earnings Calls,16:30 EST,False,False,A,Agilent Technologies Inc,/companies/A,"{'fiscal_quarter': 'Q4', 'fiscal_year': 2022, ...",NaN
4,Q3 2022 Earnings Results,2022-11-21,earnings_results,Earnings Results,None,False,False,ACAQ,Athena Consumer Acquisition Corp,/companies/ACAQ,"{'fiscal_quarter': 'Q3', 'fiscal_year': 2022, ...",NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
1659,333333 to 10000000 Split,2022-12-01,splits,Splits,None,False,False,TUEM,Tuesday Morning Corp,/companies/TUEM,{'ratio': '333333:10000000'},NaN
1660,1546061 to 10000000 Split,2022-12-01,splits,Splits,None,False,False,VSQTF,Victory Square Technologies Inc,/companies/VSQTF,{'ratio': '1546061:10000000'},NaN
1661,1546061 to 10000000 Split,2022-12-01,splits,Splits,None,False,False,VST.CX,Victory Square Technologies Inc,/companies/VST.CX,{'ratio': '1546061:10000000'},NaN
1662,208059 to 200000 Split,2022-12-01,spinoffs,Spinoffs,None,False,False,FNF,Fidelity National Financial Inc,/companies/FNF,{'ratio': '208059:200000'},NaN


`event_time_fmtd` and `currency_code` seems to have high number of `None` and `NaN` values

In [76]:
df.event_time_fmtd.isna().sum()

1441

In [77]:
df.currency_code.isna().sum()

711

From above, we can drop event_time_fmtd column since its missing large amount of data and fill NaN in currency_code column with 'Unknown' text

In [78]:
df = df.drop(['event_time_fmtd'], axis=1)
df.currency_code = df.currency_code.fillna('Unknown')
# print rows with unknown currency code
df.loc[df.currency_code=='Unknown']

,description,event_date,event_subgroup,event_subgroup_display,is_today_or_later,is_estimate,security_id,security_name,security_url,extra,currency_code
0,Q4 2022 Earnings Results,2022-11-20,earnings_results,Earnings Results,False,False,CBBYF,Virgin Money UK PLC,/companies/CBBYF,"{'fiscal_quarter': 'Q4', 'fiscal_year': 2022, ...",Unknown
1,Q4 2022 Earnings Results,2022-11-20,earnings_results,Earnings Results,False,False,CYBBF,Virgin Money UK PLC,/companies/CYBBF,"{'fiscal_quarter': 'Q4', 'fiscal_year': 2022, ...",Unknown
2,Q4 2022 Earnings Results,2022-11-21,earnings_results,Earnings Results,False,False,A,Agilent Technologies Inc,/companies/A,"{'fiscal_quarter': 'Q4', 'fiscal_year': 2022, ...",Unknown
3,Q4 2022 Earnings Call,2022-11-21,earnings_calls,Earnings Calls,False,False,A,Agilent Technologies Inc,/companies/A,"{'fiscal_quarter': 'Q4', 'fiscal_year': 2022, ...",Unknown
4,Q3 2022 Earnings Results,2022-11-21,earnings_results,Earnings Results,False,False,ACAQ,Athena Consumer Acquisition Corp,/companies/ACAQ,"{'fiscal_quarter': 'Q3', 'fiscal_year': 2022, ...",Unknown
...,...,...,...,...,...,...,...,...,...,...,...
1659,333333 to 10000000 Split,2022-12-01,splits,Splits,False,False,TUEM,Tuesday Morning Corp,/companies/TUEM,{'ratio': '333333:10000000'},Unknown
1660,1546061 to 10000000 Split,2022-12-01,splits,Splits,False,False,VSQTF,Victory Square Technologies Inc,/companies/VSQTF,{'ratio': '1546061:10000000'},Unknown
1661,1546061 to 10000000 Split,2022-12-01,splits,Splits,False,False,VST.CX,Victory Square Technologies Inc,/companies/VST.CX,{'ratio': '1546061:10000000'},Unknown
1662,208059 to 200000 Split,2022-12-01,spinoffs,Spinoffs,False,False,FNF,Fidelity National Financial Inc,/companies/FNF,{'ratio': '208059:200000'},Unknown


In [79]:
df.currency_code.unique()

array(['Unknown', 'USD', 'SEK', 'EUR', 'MXN', 'MYR', 'AUD', 'CAD', 'PHP',
       'ILS', 'SGD', 'HKD', 'ZAR', 'DKK', 'NZD', 'GBP', 'IDR', 'THB',
       'NOK', 'JPY', 'CZK'], dtype=object)

In [80]:
df.is_estimate.value_counts()

False    1663
True        1
Name: is_estimate, dtype: int64

In [81]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1664 entries, 0 to 1663
Data columns (total 11 columns):
 #   Column                  Non-Null Count  Dtype 
---  ------                  --------------  ----- 
 0   description             1664 non-null   object
 1   event_date              1664 non-null   object
 2   event_subgroup          1664 non-null   object
 3   event_subgroup_display  1664 non-null   object
 4   is_today_or_later       1664 non-null   bool  
 5   is_estimate             1664 non-null   bool  
 6   security_id             1664 non-null   object
 7   security_name           1664 non-null   object
 8   security_url            1664 non-null   object
 9   extra                   1664 non-null   object
 10  currency_code           1664 non-null   object
dtypes: bool(2), object(9)
memory usage: 120.4+ KB


`extra` column has dictionary data. Here, we are trying to find pattern in that data.

In [82]:
df.extra

0       {'fiscal_quarter': 'Q4', 'fiscal_year': 2022, ...
1       {'fiscal_quarter': 'Q4', 'fiscal_year': 2022, ...
2       {'fiscal_quarter': 'Q4', 'fiscal_year': 2022, ...
3       {'fiscal_quarter': 'Q4', 'fiscal_year': 2022, ...
4       {'fiscal_quarter': 'Q3', 'fiscal_year': 2022, ...
                              ...                        
1659                         {'ratio': '333333:10000000'}
1660                        {'ratio': '1546061:10000000'}
1661                        {'ratio': '1546061:10000000'}
1662                           {'ratio': '208059:200000'}
1663    {'fiscal_quarter': None, 'fiscal_year': 2022, ...
Name: extra, Length: 1664, dtype: object

The json data in `extras` column doesn't have common keys across all the data. We can further check if this json data is repeated/reused somewhere in dataframe

In [83]:
df.columns

Index(['description', 'event_date', 'event_subgroup', 'event_subgroup_display',
       'is_today_or_later', 'is_estimate', 'security_id', 'security_name',
       'security_url', 'extra', 'currency_code'],
      dtype='object')

In [84]:
df[['description', 'extra']]

,description,extra
0,Q4 2022 Earnings Results,"{'fiscal_quarter': 'Q4', 'fiscal_year': 2022, ..."
1,Q4 2022 Earnings Results,"{'fiscal_quarter': 'Q4', 'fiscal_year': 2022, ..."
2,Q4 2022 Earnings Results,"{'fiscal_quarter': 'Q4', 'fiscal_year': 2022, ..."
3,Q4 2022 Earnings Call,"{'fiscal_quarter': 'Q4', 'fiscal_year': 2022, ..."
4,Q3 2022 Earnings Results,"{'fiscal_quarter': 'Q3', 'fiscal_year': 2022, ..."
...,...,...
1659,333333 to 10000000 Split,{'ratio': '333333:10000000'}
1660,1546061 to 10000000 Split,{'ratio': '1546061:10000000'}
1661,1546061 to 10000000 Split,{'ratio': '1546061:10000000'}
1662,208059 to 200000 Split,{'ratio': '208059:200000'}


It is observed that the json data in `extras` column is used to create text for `description`. We can safely drop one of the 2 columns.

It is preferrred to retain `extras` column as it can be used to get better insights by storing it in json field in sql compared to a `description` string

In [85]:
df = df.drop(['description'], axis=1)

In [86]:
df

,event_date,event_subgroup,event_subgroup_display,is_today_or_later,is_estimate,security_id,security_name,security_url,extra,currency_code
0,2022-11-20,earnings_results,Earnings Results,False,False,CBBYF,Virgin Money UK PLC,/companies/CBBYF,"{'fiscal_quarter': 'Q4', 'fiscal_year': 2022, ...",Unknown
1,2022-11-20,earnings_results,Earnings Results,False,False,CYBBF,Virgin Money UK PLC,/companies/CYBBF,"{'fiscal_quarter': 'Q4', 'fiscal_year': 2022, ...",Unknown
2,2022-11-21,earnings_results,Earnings Results,False,False,A,Agilent Technologies Inc,/companies/A,"{'fiscal_quarter': 'Q4', 'fiscal_year': 2022, ...",Unknown
3,2022-11-21,earnings_calls,Earnings Calls,False,False,A,Agilent Technologies Inc,/companies/A,"{'fiscal_quarter': 'Q4', 'fiscal_year': 2022, ...",Unknown
4,2022-11-21,earnings_results,Earnings Results,False,False,ACAQ,Athena Consumer Acquisition Corp,/companies/ACAQ,"{'fiscal_quarter': 'Q3', 'fiscal_year': 2022, ...",Unknown
...,...,...,...,...,...,...,...,...,...,...
1659,2022-12-01,splits,Splits,False,False,TUEM,Tuesday Morning Corp,/companies/TUEM,{'ratio': '333333:10000000'},Unknown
1660,2022-12-01,splits,Splits,False,False,VSQTF,Victory Square Technologies Inc,/companies/VSQTF,{'ratio': '1546061:10000000'},Unknown
1661,2022-12-01,splits,Splits,False,False,VST.CX,Victory Square Technologies Inc,/companies/VST.CX,{'ratio': '1546061:10000000'},Unknown
1662,2022-12-01,spinoffs,Spinoffs,False,False,FNF,Fidelity National Financial Inc,/companies/FNF,{'ratio': '208059:200000'},Unknown


In [87]:
# check if any null data present in dataframe
df.isna().sum()

event_date                0
event_subgroup            0
event_subgroup_display    0
is_today_or_later         0
is_estimate               0
security_id               0
security_name             0
security_url              0
extra                     0
currency_code             0
dtype: int64

The above data suitable for storing in a database now with null values and unwanted columns removed

### Push Dataframe data to MS SQL
* We can use the dataframe column names and datatype to construct a schema
* Query for insert can be created by looping over data. This way we can insert  bulk data at once instead of connecting for every write to database
* The database is setup and hosted on azure virtual machine. Connection is made using pypyodbc library

In [88]:
import pypyodbc

# Connect to remote instance and create a database called `Events`

def get_db_connection(host, port, username, password, database):
  connection = pypyodbc.connect('Driver={ODBC Driver 17 for SQL Server};'
    'Server='+host+','+port+';'
    'Database='+database+';'
    'encrypt=yes;'
    'TrustServerCertificate=yes;'
    'UID='+username+';'
    'PWD='+password+';',autocommit = True)
  return connection

conn = get_db_connection('20.101.66.12','1433','sa','admin@1234','master')
try:
  cursor = conn.cursor()
  SQLCommand = ("CREATE DATABASE Events;")
  cursor.execute(SQLCommand)
  print('done')
except Exception as e:
  print('Error: '+str(e))

done


Create new table `events_table` for storing the records

In [89]:
try:
  conn = get_db_connection('20.101.66.12','1433','sa','admin@1234','Events')

  cursor = conn.cursor()
  cmd = """create table events_table ( 
  event_date                Date,
  event_subgroup            varchar(255),
  event_subgroup_display    varchar(255),
  is_today_or_later         varchar(255),
  is_estimate               varchar(255),
  security_id               varchar(255),
  security_name             varchar(255),
  security_url              varchar(255),
  extra                     nvarchar(max),
  currency_code             varchar(255)
  )"""

  cursor.execute(cmd)
  print('created table events_table')
except Exception as e:
  print('error: ',str(e))

created table events_table


we can add a constraint for JSON column so that validation happens everytime any data is added to json column

In [90]:
cmd="""
alter table events_table
    add constraint [extra record should be formatted as JSON]
                   check (ISJSON(extra)=1)
"""
cursor.execute(cmd)

**Prepare data for inserts:**

* Max Limit of `inserts` that is allowed by MSSql is 1000 per query. So, the dataframe needs to be split into chunks of 1000 or less
* The boolean and object type columns needs to be converted to string before inserting to database otherwise the database throws programming error

In following steps, the dataframe is split into chunks and inserted to database using `insert` query

In [91]:
def get_chunked_dataframe(max_row_count, dataframe):
  """
  split dataframe into n parts based on max_row_count
  returns list of dataframes
  """
  arr=[]
  chunk_size = round(len(dataframe)/max_row_count)
  for i in range(chunk_size):
    arr.append(dataframe[i*max_row_count:max_row_count*(i+1)])
  return arr

# transform object columns to str
df.is_estimate = df.is_estimate.astype(str)
df.is_today_or_later = df.is_today_or_later.astype(str)

# stringify json
df.extra = df.extra.apply(lambda x: json.dumps(x))

# remove single quotes from security_name column
df.security_name = df.security_name.apply(lambda x: x.replace("'",""))


chunked_df = get_chunked_dataframe(1000, df)

# loop over list of dataframes. for eg: if 2000 rows are present 
# in dataframe and max limit is 1000, it will loop twice

for dataframe in chunked_df:
  # convert dataframe rows into tuple list for insert query
  data_tupl_list = list(dataframe.to_records(index=False))
  # join list entries to single string for insert query
  val_str = ','.join([str(i) for i in data_tupl_list])
  print(val_str)

  cmd = 'insert into events_table values '+val_str

  cursor.execute(cmd)

print('all entries added to database')

('2022-11-20', 'earnings_results', 'Earnings Results', 'False', 'False', 'CBBYF', 'Virgin Money UK PLC', '/companies/CBBYF', '{"fiscal_quarter": "Q4", "fiscal_year": 2022, "estimate_data": null}', 'Unknown'),('2022-11-20', 'earnings_results', 'Earnings Results', 'False', 'False', 'CYBBF', 'Virgin Money UK PLC', '/companies/CYBBF', '{"fiscal_quarter": "Q4", "fiscal_year": 2022, "estimate_data": null}', 'Unknown'),('2022-11-21', 'earnings_results', 'Earnings Results', 'False', 'False', 'A', 'Agilent Technologies Inc', '/companies/A', '{"fiscal_quarter": "Q4", "fiscal_year": 2022, "estimate_data": null}', 'Unknown'),('2022-11-21', 'earnings_calls', 'Earnings Calls', 'False', 'False', 'A', 'Agilent Technologies Inc', '/companies/A', '{"fiscal_quarter": "Q4", "fiscal_year": 2022, "estimate_data": null}', 'Unknown'),('2022-11-21', 'earnings_results', 'Earnings Results', 'False', 'False', 'ACAQ', 'Athena Consumer Acquisition Corp', '/companies/ACAQ', '{"fiscal_quarter": "Q3", "fiscal_year": 2

Below, the data is being fetched and being checked. We can also fetch the JSON field values individually

In [92]:
# check count of entries in database
cursor.execute("select count(*) from events_table")
result = cursor.fetchall()
print(result)

[(1664,)]


In [93]:
conn = get_db_connection('20.101.66.12','1433','sa','admin@1234','Events')
cursor = conn.cursor()

# fetch json data from extra column
cursor.execute("""select security_name, json_value(extra, '$.fiscal_year'),
json_value(extra, '$.fiscal_quarter')
from events_table
where ISJSON(extra) > 0
  and json_value(extra, '$.fiscal_quarter') = 'Q3'
  and  event_subgroup='earnings_results'
order by event_date""")

cursor.fetchall()

[('Athena Consumer Acquisition Corp', '2022', 'Q3'),
 ('Athena Consumer Acquisition Corp (Unit)', '2022', 'Q3'),
 ('Ault Disruptive Technologies Corp', '2022', 'Q3'),
 ('AF Acquisition Corp', '2022', 'Q3'),
 ('AF Acquisition Corp (Unit)', '2022', 'Q3'),
 ('Afya Ltd', '2022', 'Q3'),
 ('Agora Inc', '2022', 'Q3'),
 ('AxonPrime Infrastructure Acquisition Corp', '2022', 'Q3'),
 ('AxonPrime Infrastructure Acquisition Corp (Unit)', '2022', 'Q3'),
 ('BlueRiver Acquisition Corp', '2022', 'Q3'),
 ('BlueRiver Acquisition Corp (Unit)', '2022', 'Q3'),
 ('Can B Corp', '2022', 'Q3'),
 ('CENAQ Energy Corp', '2022', 'Q3'),
 ('CENAQ Energy Corp (Unit)', '2022', 'Q3'),
 ('Colonnade Acquisition Corp II', '2022', 'Q3'),
 ('Colonnade Acquisition Corp II (Unit)', '2022', 'Q3'),
 ('Compute Health Acquisition Corp', '2022', 'Q3'),
 ('Compute Health Acquisition Corp (Unit)', '2022', 'Q3'),
 ('CPI Aerostructures Inc', '2022', 'Q3'),
 ('Dell Technologies Inc', '2023', 'Q3'),
 ('DouYu International Holdings Ltd', 

In [94]:
cursor.execute("select top 1 * from events_table where event_date='2022-12-01'")
data = cursor.fetchall()
print(data)
cursor.close()

[(datetime.date(2022, 12, 1), 'earnings_results', 'Earnings Results', 'False', 'False', 'AOUT', 'American Outdoor Brands Inc', '/companies/AOUT', '{"fiscal_quarter": "Q2", "fiscal_year": 2023, "estimate_data": null}', 'Unknown')]


In [ ]:
# used in case re running code blocks
# conn = get_db_connection('20.101.66.12','1433','sa','admin@1234','master')
# cursor = conn.cursor()
# cursor.execute('DROP database Events;')